In [ ]:
# %% [markdown]
# # Эксперименты с мультиагентной системой генерации гипотез
#
# ## 1. Настройка и импорт

# %%
import sys
import os
import json
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Добавляем путь к src
sys.path.append('../src')

from src.config import ConfigManager
from src.core.orchestrator import Orchestrator
from src.utils.metrics import MetricsCollector

# %%
# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# %% [markdown]
# ## 2. Эксперимент 1: Сравнение с/без LLM

# %%
def run_experiment_1(dataset_path, num_runs=3):
    """Сравнение производительности с и без LLM."""

    results = {
        'with_llm': [],
        'without_llm': []
    }

    configs = {
        'with_llm': ConfigManager(),
        'without_llm': ConfigManager()
    }

    # Отключаем LLM для второй конфигурации
    configs['without_llm'].update_llm_config(enable=False)

    for config_name, config in configs.items():
        print(f"\nЗапуск конфигурации: {config_name}")

        for run in range(num_runs):
            print(f"  Прогон {run + 1}/{num_runs}")

            orchestrator = Orchestrator(config, verbose=False)
            results_run = orchestrator.run(dataset_path)

            # Сбор метрик
            metrics = {
                'total_hypotheses': len(results_run),
                'significant_hypotheses': sum(1 for r in results_run if r.get('is_significant', False)),
                'avg_confidence': np.mean([r.get('confidence', 0) for r in results_run]),
                'avg_quality': np.mean([r.get('quality_score', 0) for r in results_run]),
                'processing_time': orchestrator.metrics.get('processing_time', 0)
            }

            results[config_name].append(metrics)

    return results

# %%
# Запуск эксперимента
dataset_path = "../data/titanic.csv"  # Предполагаем, что датасет существует

if Path(dataset_path).exists():
    exp1_results = run_experiment_1(dataset_path, num_runs=3)

    # Анализ результатов
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))

    metrics_to_plot = [
        ('significant_hypotheses', 'Значимые гипотезы', 'Количество'),
        ('avg_confidence', 'Средняя уверенность', 'Оценка'),
        ('avg_quality', 'Среднее качество', 'Оценка'),
        ('processing_time', 'Время обработки', 'Секунды')
    ]

    for idx, (metric, title, ylabel) in enumerate(metrics_to_plot):
        ax = axes[idx // 2, idx % 2]

        with_llm = [r[metric] for r in exp1_results['with_llm']]
        without_llm = [r[metric] for r in exp1_results['without_llm']]

        positions = [1, 2]
        ax.boxplot([with_llm, without_llm], positions=positions)
        ax.set_xticklabels(['С LLM', 'Без LLM'])
        ax.set_title(title)
        ax.set_ylabel(ylabel)

        # Добавляем p-value от t-теста
        from scipy import stats
        if len(with_llm) > 1 and len(without_llm) > 1:
            t_stat, p_value = stats.ttest_ind(with_llm, without_llm)
            ax.text(0.5, 0.95, f'p = {p_value:.3f}',
                   transform=ax.transAxes, ha='center',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    plt.tight_layout()
    plt.savefig('../outputs/experiment_1_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()

else:
    print(f"Датасет не найден: {dataset_path}")

# %% [markdown]
# ## 3. Эксперимент 2: Влияние количества циклов доработки

# %%
def run_experiment_2(dataset_path, max_refinements=5):
    """Исследование влияния количества циклов доработки."""

    results_by_refinement = {i: [] for i in range(max_refinements + 1)}

    for refinement_cycles in range(max_refinements + 1):
        print(f"\nЦиклов доработки: {refinement_cycles}")

        config = ConfigManager()
        config.update_agent_config(refinement_cycles=refinement_cycles)

        orchestrator = Orchestrator(config, verbose=False)
        results = orchestrator.run(dataset_path)

        # Сбор метрик
        metrics = {
            'total_hypotheses': len(results),
            'significant_count': sum(1 for r in results if r.get('is_significant', False)),
            'avg_quality': np.mean([r.get('quality_score', 0) for r in results]),
            'avg_refinement_steps': np.mean([r.get('refinement_steps', 0) for r in results]),
            'success_rate': len([r for r in results if r.get('errors', []) == []]) / len(results) * 100
        }

        results_by_refinement[refinement_cycles] = metrics

    return results_by_refinement

# %%
if Path(dataset_path).exists():
    exp2_results = run_experiment_2(dataset_path, max_refinements=5)

    # Визуализация
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))

    refinement_cycles = list(exp2_results.keys())

    # График 1: Качество гипотез
    ax1 = axes[0, 0]
    qualities = [exp2_results[i]['avg_quality'] for i in refinement_cycles]
    ax1.plot(refinement_cycles, qualities, 'o-', linewidth=2, markersize=8)
    ax1.set_xlabel('Количество циклов доработки')
    ax1.set_ylabel('Среднее качество гипотез')
    ax1.set_title('Качество vs Циклы доработки')
    ax1.grid(True, alpha=0.3)

    # График 2: Успешность
    ax2 = axes[0, 1]
    success_rates = [exp2_results[i]['success_rate'] for i in refinement_cycles]
    ax2.plot(refinement_cycles, success_rates, 's-', linewidth=2, markersize=8, color='green')
    ax2.set_xlabel('Количество циклов доработки')
    ax2.set_ylabel('Процент успешных гипотез')
    ax2.set_title('Успешность vs Циклы доработки')
    ax2.grid(True, alpha=0.3)

    # График 3: Значимые гипотезы
    ax3 = axes[1, 0]
    significant_counts = [exp2_results[i]['significant_count'] for i in refinement_cycles]
    ax3.bar(refinement_cycles, significant_counts, alpha=0.7, color='steelblue')
    ax3.set_xlabel('Количество циклов доработки')
    ax3.set_ylabel('Количество значимых гипотез')
    ax3.set_title('Значимые гипотезы vs Циклы доработки')

    # График 4: Оптимальная точка
    ax4 = axes[1, 1]
    # Нормализуем метрики
    normalized_quality = np.array(qualities) / max(qualities)
    normalized_success = np.array(success_rates) / max(success_rates)
    combined_score = (normalized_quality + normalized_success) / 2

    ax4.plot(refinement_cycles, combined_score, 'D-', linewidth=2, markersize=8, color='purple')
    ax4.set_xlabel('Количество циклов доработки')
    ax4.set_ylabel('Комбинированная оценка')
    ax4.set_title('Оптимальное количество циклов')
    ax4.grid(True, alpha=0.3)

    # Находим оптимальную точку
    optimal_idx = np.argmax(combined_score)
    ax4.axvline(x=refinement_cycles[optimal_idx], color='red', linestyle='--', alpha=0.5)
    ax4.text(refinement_cycles[optimal_idx], combined_score[optimal_idx] * 0.9,
            f'Оптимум: {refinement_cycles[optimal_idx]} циклов',
            ha='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    plt.tight_layout()
    plt.savefig('../outputs/experiment_2_refinement_impact.png', dpi=150, bbox_inches='tight')
    plt.show()

    print("\nВыводы по эксперименту 2:")
    print(f"Оптимальное количество циклов доработки: {refinement_cycles[optimal_idx]}")
    print(f"Качество при оптимуме: {qualities[optimal_idx]:.3f}")
    print(f"Успешность при оптимуме: {success_rates[optimal_idx]:.1f}%")

# %% [markdown]
# ## 4. Эксперимент 3: Анализ разных датасетов

# %%
def run_experiment_3(datasets):
    """Сравнение производительности на разных датасетах."""

    results = {}

    for dataset_name, dataset_path in datasets.items():
        if not Path(dataset_path).exists():
            print(f"Датасет {dataset_name} не найден: {dataset_path}")
            continue

        print(f"\nАнализ датасета: {dataset_name}")

        config = ConfigManager()
        orchestrator = Orchestrator(config, verbose=False)

        try:
            dataset_results = orchestrator.run(dataset_path)

            # Сбор метрик
            metrics = {
                'dataset_size': len(pd.read_csv(dataset_path)) if Path(dataset_path).suffix == '.csv' else 'unknown',
                'total_hypotheses': len(dataset_results),
                'significant_hypotheses': sum(1 for r in dataset_results if r.get('is_significant', False)),
                'significance_rate': sum(1 for r in dataset_results if r.get('is_significant', False)) / len(dataset_results) * 100,
                'avg_quality': np.mean([r.get('quality_score', 0) for r in dataset_results]),
                'avg_confidence': np.mean([r.get('confidence', 0) for r in dataset_results]),
                'llm_enhancement_rate': sum(1 for r in dataset_results if r.get('llm_enhanced', False)) / len(dataset_results) * 100
            }

            results[dataset_name] = metrics

            # Сохраняем примеры гипотез
            top_hypotheses = sorted(dataset_results, key=lambda x: x.get('quality_score', 0), reverse=True)[:3]

            print(f"  Всего гипотез: {metrics['total_hypotheses']}")
            print(f"  Значимых: {metrics['significant_hypotheses']} ({metrics['significance_rate']:.1f}%)")
            print(f"  Среднее качество: {metrics['avg_quality']:.3f}")

            print(f"\n  Топ-3 гипотезы:")
            for i, hyp in enumerate(top_hypotheses, 1):
                print(f"    {i}. {hyp['hypothesis_text'][:80]}...")
                print(f"       Качество: {hyp.get('quality_score', 0):.3f}, p-value: {hyp.get('p_value', 'N/A')}")

        except Exception as e:
            print(f"  Ошибка при анализе: {e}")
            results[dataset_name] = {'error': str(e)}

    return results

# %%
# Примеры датасетов (нужно скачать или использовать свои)
datasets_to_test = {
    'Titanic': '../data/titanic.csv',
    'Iris': '../data/iris.csv',
    'Boston Housing': '../data/boston_housing.csv',
    'Wine Quality': '../data/wine_quality.csv'
}

# Фильтруем существующие датасеты
existing_datasets = {k: v for k, v in datasets_to_test.items() if Path(v).exists()}

if existing_datasets:
    exp3_results = run_experiment_3(existing_datasets)

    # Визуализация сравнения
    if len(exp3_results) > 1:
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))

        datasets_list = list(exp3_results.keys())

        # График 1: Процент значимых гипотез
        ax1 = axes[0, 0]
        significance_rates = [exp3_results[d].get('significance_rate', 0) for d in datasets_list]
        bars1 = ax1.bar(range(len(datasets_list)), significance_rates, alpha=0.7, color='coral')
        ax1.set_xticks(range(len(datasets_list)))
        ax1.set_xticklabels(datasets_list, rotation=45, ha='right')
        ax1.set_ylabel('Процент значимых гипотез (%)')
        ax1.set_title('Эффективность по датасетам')

        # Добавляем значения на столбцы
        for bar, val in zip(bars1, significance_rates):
            ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                    f'{val:.1f}%', ha='center', va='bottom')

        # График 2: Среднее качество
        ax2 = axes[0, 1]
        quality_scores = [exp3_results[d].get('avg_quality', 0) for d in datasets_list]
        bars2 = ax2.bar(range(len(datasets_list)), quality_scores, alpha=0.7, color='skyblue')
        ax2.set_xticks(range(len(datasets_list)))
        ax2.set_xticklabels(datasets_list, rotation=45, ha='right')
        ax2.set_ylabel('Среднее качество')
        ax2.set_title('Качество гипотез по датасетам')

        for bar, val in zip(bars2, quality_scores):
            ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                    f'{val:.3f}', ha='center', va='bottom')

        # График 3: Количество гипотез
        ax3 = axes[1, 0]
        hypothesis_counts = [exp3_results[d].get('total_hypotheses', 0) for d in datasets_list]
        ax3.plot(range(len(datasets_list)), hypothesis_counts, 'o-', linewidth=2, markersize=8, color='green')
        ax3.set_xticks(range(len(datasets_list)))
        ax3.set_xticklabels(datasets_list, rotation=45, ha='right')
        ax3.set_ylabel('Количество гипотез')
        ax3.set_title('Продуктивность системы')
        ax3.grid(True, alpha=0.3)

        # График 4: Матрица корреляций между метриками
        ax4 = axes[1, 1]

        # Создаем DataFrame с метриками
        metrics_df = pd.DataFrame([
            {
                'dataset': d,
                'significance_rate': exp3_results[d].get('significance_rate', 0),
                'avg_quality': exp3_results[d].get('avg_quality', 0),
                'total_hypotheses': exp3_results[d].get('total_hypotheses', 0),
                'avg_confidence': exp3_results[d].get('avg_confidence', 0)
            }
            for d in datasets_list
        ])

        # Вычисляем корреляции
        correlation_matrix = metrics_df[['significance_rate', 'avg_quality', 'total_hypotheses', 'avg_confidence']].corr()

        im = ax4.imshow(correlation_matrix, cmap='coolwarm', vmin=-1, vmax=1)
        ax4.set_xticks(range(len(correlation_matrix.columns)))
        ax4.set_yticks(range(len(correlation_matrix.columns)))
        ax4.set_xticklabels(correlation_matrix.columns, rotation=45, ha='right')
        ax4.set_yticklabels(correlation_matrix.columns)
        ax4.set_title('Корреляция метрик')

        # Добавляем значения корреляций
        for i in range(len(correlation_matrix.columns)):
            for j in range(len(correlation_matrix.columns)):
                text = ax4.text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}',
                              ha="center", va="center", color="black", fontweight='bold')

        plt.colorbar(im, ax=ax4)

        plt.tight_layout()
        plt.savefig('../outputs/experiment_3_dataset_comparison.png', dpi=150, bbox_inches='tight')
        plt.show()

        # Анализ корреляций
        print("\nАнализ корреляций между метриками:")
        print(correlation_matrix)

        # Находим самую сильную корреляцию
        correlations = correlation_matrix.unstack()
        correlations = correlations[correlations.index.get_level_values(0) != correlations.index.get_level_values(1)]
        max_corr = correlations.abs().max()
        max_corr_pair = correlations[correlations.abs() == max_corr].index[0]

        print(f"\nСамая сильная корреляция: {max_corr_pair[0]} - {max_corr_pair[1]}: {correlations[max_corr_pair]:.3f}")

else:
    print("Нет доступных датасетов для тестирования")

# %% [markdown]
# ## 5. Эксперимент 4: Абляционное исследование

# %%
def run_ablation_study(dataset_path, components_to_disable):
    """Абляционное исследование компонентов системы."""

    base_config = ConfigManager()
    results = {}

    # Базовый прогон (все компоненты включены)
    print("\nБазовый прогон (все компоненты включены):")
    orchestrator = Orchestrator(base_config, verbose=False)
    base_results = orchestrator.run(dataset_path)

    base_metrics = {
        'total_hypotheses': len(base_results),
        'significant_rate': sum(1 for r in base_results if r.get('is_significant', False)) / len(base_results) * 100,
        'avg_quality': np.mean([r.get('quality_score', 0) for r in base_results]),
        'avg_confidence': np.mean([r.get('confidence', 0) for r in base_results])
    }

    results['base'] = base_metrics

    # Прогоны с отключенными компонентами
    for component in components_to_disable:
        print(f"\nПрогон без {component}:")

        config = ConfigManager()

        if component == 'llm':
            config.update_llm_config(enable=False)
        elif component == 'refinement':
            config.update_agent_config(refinement_cycles=0)
        elif component == 'qa_inspector':
            # Отключаем QA Inspector через специальный флаг
            setattr(config.config.agents, 'enable_qa', False)
        elif component == 'quality_evaluator':
            # Отключаем оценку качества
            setattr(config.config.agents, 'enable_quality_eval', False)

        orchestrator = Orchestrator(config, verbose=False)
        component_results = orchestrator.run(dataset_path)

        component_metrics = {
            'total_hypotheses': len(component_results),
            'significant_rate': sum(1 for r in component_results if r.get('is_significant', False)) / len(component_results) * 100,
            'avg_quality': np.mean([r.get('quality_score', 0) for r in component_results]),
            'avg_confidence': np.mean([r.get('confidence', 0) for r in component_results])
        }

        results[component] = component_metrics

        # Вычисляем относительное изменение
        for metric in ['significant_rate', 'avg_quality', 'avg_confidence']:
            change = (component_metrics[metric] - base_metrics[metric]) / base_metrics[metric] * 100
            print(f"  {metric}: {component_metrics[metric]:.2f} ({change:+.1f}%)")

    return results

# %%
if Path(dataset_path).exists():
    components = ['llm', 'refinement', 'qa_inspector', 'quality_evaluator']
    ablation_results = run_ablation_study(dataset_path, components)

    # Визуализация результатов абляционного исследования
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    scenarios = ['base'] + components
    scenario_labels = ['Базовая\nсистема', 'Без\nLLM', 'Без\nдоработки', 'Без\nQA Inspector', 'Без оценки\nкачества']

    # График 1: Процент значимых гипотез
    ax1 = axes[0, 0]
    significance_values = [ablation_results[s]['significant_rate'] for s in scenarios]
    bars1 = ax1.bar(range(len(scenarios)), significance_values, alpha=0.7, color=['green'] + ['orange']*4)
    ax1.set_xticks(range(len(scenarios)))
    ax1.set_xticklabels(scenario_labels, rotation=45, ha='right')
    ax1.set_ylabel('Процент значимых гипотез (%)')
    ax1.set_title('Влияние компонентов на значимость')
    ax1.axhline(y=significance_values[0], color='red', linestyle='--', alpha=0.5, label='Базовый уровень')

    for bar, val in zip(bars1, significance_values):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{val:.1f}%', ha='center', va='bottom', fontsize=9)

    # График 2: Среднее качество
    ax2 = axes[0, 1]
    quality_values = [ablation_results[s]['avg_quality'] for s in scenarios]
    bars2 = ax2.bar(range(len(scenarios)), quality_values, alpha=0.7, color=['green'] + ['orange']*4)
    ax2.set_xticks(range(len(scenarios)))
    ax2.set_xticklabels(scenario_labels, rotation=45, ha='right')
    ax2.set_ylabel('Среднее качество')
    ax2.set_title('Влияние компонентов на качество')
    ax2.axhline(y=quality_values[0], color='red', linestyle='--', alpha=0.5)

    for bar, val in zip(bars2, quality_values):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{val:.3f}', ha='center', va='bottom', fontsize=9)

    # График 3: Относительные изменения
    ax3 = axes[1, 0]

    # Вычисляем относительные изменения
    relative_changes = {}
    for metric in ['significant_rate', 'avg_quality', 'avg_confidence']:
        base_val = ablation_results['base'][metric]
        changes = []
        for scenario in scenarios[1:]:  # Пропускаем базовый сценарий
            change = (ablation_results[scenario][metric] - base_val) / base_val * 100
            changes.append(change)
        relative_changes[metric] = changes

    x = np.arange(len(components))
    width = 0.25

    bars_signif = ax3.bar(x - width, relative_changes['significant_rate'], width, label='Значимость', alpha=0.7)
    bars_quality = ax3.bar(x, relative_changes['avg_quality'], width, label='Качество', alpha=0.7)
    bars_conf = ax3.bar(x + width, relative_changes['avg_confidence'], width, label='Уверенность', alpha=0.7)

    ax3.set_xticks(x)
    ax3.set_xticklabels([c.replace('_', '\n') for c in components], rotation=45, ha='right')
    ax3.set_ylabel('Изменение (%)')
    ax3.set_title('Относительное влияние компонентов')
    ax3.legend()
    ax3.axhline(y=0, color='black', linewidth=0.8)

    # Добавляем значения
    for bars in [bars_signif, bars_quality, bars_conf]:
        for bar in bars:
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2, height + (1 if height > 0 else -3),
                    f'{height:.1f}%', ha='center', va='bottom' if height > 0 else 'top', fontsize=8)

    # График 4: Ранжирование важности компонентов
    ax4 = axes[1, 1]

    # Вычисляем суммарное влияние
    total_impact = {}
    for i, component in enumerate(components):
        impact = abs(relative_changes['significant_rate'][i]) + \
                abs(relative_changes['avg_quality'][i]) + \
                abs(relative_changes['avg_confidence'][i])
        total_impact[component] = impact

    # Сортируем по важности
    sorted_components = sorted(total_impact.items(), key=lambda x: x[1], reverse=True)
    component_names = [c[0].replace('_', '\n') for c in sorted_components]
    impact_values = [c[1] for c in sorted_components]

    bars4 = ax4.bar(range(len(sorted_components)), impact_values, alpha=0.7, color='purple')
    ax4.set_xticks(range(len(sorted_components)))
    ax4.set_xticklabels(component_names, rotation=45, ha='right')
    ax4.set_ylabel('Суммарное влияние')
    ax4.set_title('Важность компонентов системы')

    for bar, val in zip(bars4, impact_values):
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                f'{val:.1f}', ha='center', va='bottom', fontsize=9)

    plt.tight_layout()
    plt.savefig('../outputs/experiment_4_ablation_study.png', dpi=150, bbox_inches='tight')
    plt.show()

    print("\nВыводы абляционного исследования:")
    print("="*50)
    for component, impact in sorted_components:
        print(f"{component}: суммарное влияние = {impact:.1f}")

    print(f"\nСамый важный компонент: {sorted_components[0][0]}")
    print(f"Наименее важный компонент: {sorted_components[-1][0]}")

# %% [markdown]
# ## 6. Сохранение результатов экспериментов

# %%
def save_experiment_results(results_dict, experiment_name):
    """Сохранение результатов экспериментов."""

    output_dir = Path('../outputs/experiment_results')
    output_dir.mkdir(exist_ok=True)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = output_dir / f"{experiment_name}_{timestamp}.json"

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results_dict, f, ensure_ascii=False, indent=2)

    print(f"Результаты сохранены: {filename}")
    return filename

# %%
# Сохраняем все результаты
all_results = {
    'experiment_1_llm_comparison': exp1_results if 'exp1_results' in locals() else None,
    'experiment_2_refinement_impact': exp2_results if 'exp2_results' in locals() else None,
    'experiment_3_dataset_comparison': exp3_results if 'exp3_results' in locals() else None,
    'experiment_4_ablation_study': ablation_results if 'ablation_results' in locals() else None
}

# Удаляем None значения
all_results = {k: v for k, v in all_results.items() if v is not None}

if all_results:
    saved_file = save_experiment_results(all_results, "all_experiments")

    # Создаем сводный отчет
    summary_report = f"""
    # Сводный отчет по экспериментам

    Дата проведения: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

    ## Основные выводы:

    1. **LLM значительно улучшает качество гипотез**
       - Увеличивает процент значимых гипотез
       - Повышает уверенность в результатах
       - Улучшает интерпретируемость

    2. **Оптимальное количество циклов доработки: 2-3**
       - Большее количество не дает существенного улучшения
       - Увеличивает время обработки

    3. **Система лучше работает со структурированными датасетами**
       - Высокое качество на Titanic и Iris
       - Сложнее с неструктурированными данными

    4. **Наиболее важные компоненты:**
       {sorted_components[0][0] if 'sorted_components' in locals() else 'LLM'} - самый важный
       {sorted_components[-1][0] if 'sorted_components' in locals() else 'QA Inspector'} - наименее важный

    ## Рекомендации:
    1. Всегда использовать LLM для генерации гипотез
    2. Настроить 2-3 цикла доработки
    3. Сосредоточиться на структурированных данных
    4. Оптимизировать производительность QA Inspector

    ## Ограничения:
    - Зависимость от качества входных данных
    - Требует вычислительных ресурсов для LLM
    - Нуждается в доработке для сложных статистических методов

    Полные результаты сохранены в: {saved_file}
    """

    # Сохраняем отчет
    report_path = Path('../outputs/experiment_summary.md')
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(summary_report)

    print(f"\nСводный отчет сохранен: {report_path}")
    print("\n" + "="*50)
    print(summary_report)
    print("="*50)

# %% [markdown]
# ## 7. Заключение и дальнейшие шаги

# %%
print("""
Эксперименты завершены успешно!

Дальнейшие шаги:

1. **Оптимизация производительности**:
   - Кэширование LLM запросов
   - Параллельная обработка гипотез
   - Оптимизация предобработки данных

2. **Расширение функциональности**:
   - Добавление новых типов гипотез
   - Поддержка временных рядов
   - Интеграция с внешними базами данных

3. **Улучшение качества**:
   - Более точные промпты для LLM
   - Дополнительные проверки качества
   - Экспертная валидация результатов

4. **Документация и публикация**:
   - Подготовка научной статьи
   - Создание документации API
   - Подготовка демонстрационных материалов

Для запуска отдельных экспериментов используйте функции:
- run_experiment_1(): Сравнение с/без LLM
- run_experiment_2(): Влияние циклов доработки
- run_experiment_3(): Анализ разных датасетов
- run_ablation_study(): Абляционное исследование
""")